In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from bisect import bisect
import operator

In [2]:
def read_data(file_name):
    file_path = '../FallCatcher_Data/' + file_name
    data = pd.read_csv(file_path,sep=',',names = ['time_counter','accel_base_X','accel_base_Y','accel_base_Z','gyro_base_X',
                                                  'gyro_base_Y','gyro_base_Z','accel_right_X','accel_right_Y','accel_right_Z',
                                                  'gyro_right_X','gyro_right_Y','gyro_right_Z','accel_left_X','accel_left_Y',
                                                  'accel_left_Z','gyro_left_X','gyro_left_Y','gyro_left_Z','fall_detection',
                                                  'fall_check','impact_value'])
    return data

In [3]:
def read_combined_data(file_name,airbag_data):
    file_path = '../FallCatcher_Data/Combined/'+file_name
    data = pd.read_excel(file_path, sheet_name=0)
    data['Time Point Name'] = pd.to_numeric(data['Time Point Name'], errors='coerce')
    data = data[(data['Time Point Name']>100) & (data['Time Point Name'] < 200) &
                (data['Impact milisec'].notnull()) & ((data['Start milisec']) < (airbag_data.iloc[-1][0]))]
    data = data[['Time Point Name','Start milisec','Impact milisec', 'End Milisec','Duration']]
    return data

In [10]:
def add_new_fall_value(airbag_data,manual_data):
    airbag_data['accel_base_mag'] =  np.sqrt(airbag_data['accel_base_X']**2 + airbag_data['accel_base_Y']**2 + 
                                             airbag_data['accel_base_Z']**2)
    for index, row in manual_data.iterrows():
        airbag_data.loc[(airbag_data['time_counter'] >= row['Start milisec']) &
                        (airbag_data['time_counter'] < row['End Milisec']),'fall_check'] = 88
        airbag_data.loc[(airbag_data['accel_base_mag'] == \
                                                  (airbag_data.loc[(airbag_data['time_counter'] >= row['Start milisec']) & \
                                                                   (airbag_data['time_counter'] < row['End Milisec']),
                                                                   'accel_base_mag'].max())) & \
                                                 (airbag_data['time_counter'] >= row['Start milisec']) & \
                                                 (airbag_data['time_counter'] < row['End Milisec']),
                                                 'cal_impact_milisec'] = 80
        if(any(airbag_data['time_counter'] == row['Impact milisec'])):
            airbag_data.loc[(airbag_data['time_counter'] == row['Impact milisec']),'impact_value'] =  84
        else:
            airbag_data.loc[bisect(airbag_data['time_counter'].values, row['Impact milisec']),'impact_value'] =  84

In [50]:
def old_extract_feature(data,window_time=5000):
    max_ms = data['time_counter'].max()
    feature = pd.DataFrame(columns={'time_counter','min_acc','max_acc','fall_value'})
    for index in range(0,max_ms,window_time):
        end_index = index + window_time
        
        time_counter = data.loc[(data['time_counter'] >= index) & (data['time_counter'] < end_index),'time_counter'].min()
        min_acc = data.loc[(data['time_counter'] >= index) & (data['time_counter'] < end_index),'accel_base_mag'].min()
        max_acc = data.loc[(data['time_counter'] >= index) & (data['time_counter'] < end_index),'accel_base_mag'].max()
        if(data.loc[(data['time_counter'] > index) & (data['time_counter'] < end_index)&
                    (data['fall_check']==88),'fall_check'].any()):
            fall_value = 1
        else : 
            fall_value = 0
            
        feature = feature.append({'time_counter':time_counter,'min_acc':min_acc,'max_acc':max_acc,'fall_value':fall_value},
                       ignore_index = True)
    
    return feature

In [46]:
def extract_feature(data,window_time=4000):
    
    max_ms = data['time_counter'].max()
    feature = pd.DataFrame(columns={'time_counter','min_acc','max_acc','fall_value'})
    impact_list = data.loc[(data['cal_impact_milisec'] == 80),'time_counter'].values
    
    end_list = [0]
    start_list = []
    for index in impact_list:
        end_list.append(index + int(window_time/2))
        start_list.append(index - int(window_time/2))
    start_list.append(max_ms)
    
    #add fall values
    for index in impact_list:
        start = index - window_time/2
        end = index + window_time/2
        
        time_counter = data.loc[(data['time_counter'] >= start) & (data['time_counter'] < end),'time_counter'].min()
        min_acc = data.loc[(data['time_counter'] >= start) & (data['time_counter'] < end),'accel_base_mag'].min()
        max_acc = data.loc[(data['time_counter'] >= start) & (data['time_counter'] < end),'accel_base_mag'].max()
        fall_value = 1
        feature = feature.append({'time_counter':time_counter,'min_acc':min_acc,'max_acc':max_acc,'fall_value':fall_value},
                       ignore_index = True)
        
    #add non-fall values
    for start,end in zip(end_list,start_list):
        for index in range(start,end,window_time):
            end_index = index + window_time
            if end_index < end:
                time_counter = data.loc[(data['time_counter'] >= index) & (data['time_counter'] < end_index),'time_counter'].min()
                min_acc = data.loc[(data['time_counter'] >= index) & (data['time_counter'] < end_index),'accel_base_mag'].min()
                max_acc = data.loc[(data['time_counter'] >= index) & (data['time_counter'] < end_index),'accel_base_mag'].max()
                fall_value = 0
                feature = feature.append({'time_counter':time_counter,'min_acc':min_acc,'max_acc':max_acc,'fall_value':fall_value},
                                         ignore_index = True)

    return feature

# For S1 

In [11]:
s1_data=read_data('Airbag-S1.CSV')
s1_combined = read_combined_data('S1-combined.xlsx',s1_data)
add_new_fall_value(s1_data,s1_combined)

In [47]:
s1_feature = extract_feature(s1_data)

In [54]:
s1_feature.to_excel('S1_feature.xlsx')

In [ ]:
listv = s1_data.loc[(s1_data['cal_impact_milisec'] == 80),'time_counter'].values
print (listv)
s1_data.loc[(s1_data['time_counter'] > 691618) & (s1_data['time_counter'] < 700000)&
            (s1_data['fall_check']==88),'fall_check'].any()
s1_feature.head()
s1_feature.loc[s1_feature['fall_value']==1]
s1_feature.loc[s1_feature['fall_value']==0]

# For S2

In [59]:
s2_data=read_data('Airbag-S2.CSV')
s2_combined = read_combined_data('S2-combined.xlsx',s2_data)
add_new_fall_value(s2_data,s2_combined)

In [60]:
s2_feature = extract_feature(s2_data)

In [61]:
s2_feature.to_excel('S2_feature.xlsx')

# For S3

In [63]:
s3_data=read_data('Airbag-S3.CSV')
s3_combined = read_combined_data('S3-combined.xlsx',s3_data)
add_new_fall_value(s3_data,s3_combined)

In [64]:
s3_feature = extract_feature(s3_data)

In [65]:
s3_feature.to_excel('S3_feature.xlsx')

# For C1

In [66]:
c1_data=read_data('C1-Airbag.CSV')
c1_combined = read_combined_data('C1-combined.xlsx',c1_data)
add_new_fall_value(c1_data,c1_combined)

In [67]:
c1_feature = extract_feature(c1_data)

In [68]:
c1_feature.to_excel('C1_feature.xlsx')

# For C2

In [71]:
c2_data=read_data('C2-Airbag.CSV')
c2_combined = read_combined_data('C2-combined.xlsx',c2_data)
add_new_fall_value(c2_data,c2_combined)

In [72]:
c2_feature = extract_feature(c2_data)

In [73]:
c2_feature.to_excel('C2_feature.xlsx')

# For C3

In [75]:
c3_data=read_data('C3-Airbag.CSV')
c3_combined = read_combined_data('C3-combined.xlsx',c3_data)
add_new_fall_value(c3_data,c3_combined)

In [76]:
c3_feature = extract_feature(c3_data)

In [77]:
c3_feature.to_excel('C3_feature.xlsx')